In [31]:
!pip install wikipedia setuptools pywikibot mwparserfromhell pandas numpy scipy nltk tqdm seaborn pyqt5 pyqtwebengine ruamel-yaml lxml wikipedia-api

ERROR: Could not find a version that satisfies the requirement difflib (from versions: none)
ERROR: No matching distribution found for difflib


In [32]:
import wikipedia
import re
import pywikibot
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from urllib.request import urlopen
import json
import wikipediaapi
import difflib

#### Retrieving names of sections in an article

In [42]:
# Create a custom user-agent string
headers = {
    'User-Agent': 'YourAppName/1.0 (berenis0102@gmail.com)'  # Replace with your app name and contact info
}

# Initialize the Wikipedia API with a custom user-agent
wiki = wikipediaapi.Wikipedia(
    'en',  # 'en' for English Wikipedia
    headers=headers
)

# Get the page object for the article
page = wiki.page('Pushkin')  # Replace with the article title

# Function to recursively list sections
def print_sections(sections, level=0):
    for section in sections:
        print("%s: %s" % ("*" * (level + 1), section.title))
        print_sections(section.sections, level + 1)

# List sections of the page
print_sections(page.sections)

*: Ancestry
*: Early life
*: Social activism
*: Rise
*: Death
*: Descendants
*: Legacy
**: Literary
**: Musical
**: Romanticism
**: Russian literature
**: Soviet centennial celebrations
*: Honours
*: Works
**: Narrative poems
**: Drama
**: Fairy tales in verse
**: Short poems
**: Novels
**: Short stories
**: Non-fiction
*: See also
*: Notes
*: References
*: Further reading
*: External links


# Retrieving content of a particular section

In [37]:
def get_section_content(sections, section_title):
    for section in sections:
        if section.title == section_title:
            return section.text
        sub_section = get_section_content(section.sections, section_title)
        if sub_section:
            return sub_section
    return None

section_text = get_section_content(page.sections, 'Cough')
section_text

'Codeine was once viewed as the "gold standard" in cough suppressants, but this position is now questioned. Some recent placebo-controlled trials have found that it may be no better than a placebo for some causes including acute cough in children. As a consequence, it is not recommended for children. Additionally, there is no evidence that hydrocodone is useful in children. Similarly, a 2012 Dutch guideline regarding the treatment of acute cough does not recommend its use. (The opioid analogue dextromethorphan, long claimed to be as effective a cough suppressant as codeine, has similarly demonstrated little benefit in several recent studies.)\nLow dose morphine may help chronic cough but its use is limited by side effects.'

#### Retrieving all the variants of text of a section and all the differences between them

In [38]:
# Define the MediaWiki API URL
url = "https://en.wikipedia.org/w/api.php"

# Function to extract a specific section from wikitext using a regex
def extract_section(wikitext, section_title):
    # Use regex to find the section and its content
    section_pattern = re.compile(r"==\s*" + re.escape(section_title) + r"\s*==\s*(.*?)\s*(?=^==|\Z)", re.DOTALL | re.MULTILINE)
    match = section_pattern.search(wikitext)
    if match:
        return match.group(1).strip()
    return None

# Function to display differences between two versions using difflib
def display_diff(text1, text2):
    diff = difflib.unified_diff(
        text1.splitlines(), text2.splitlines(),
        lineterm='', n=2  # Display context lines
    )
    return '\n'.join(diff)

# Function to get all revisions of the page
def get_revisions(page_title, section_title):
    # Initialize params for the API request
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": page_title,
        "rvprop": "ids|timestamp|user|comment|content",
        "rvslots": "main",  # Ensures we get the content from the main slot
        "rvlimit": "max",  # Get all available revisions
        "rvdir": "newer"  # Start from the oldest revisions
    }
    
    # Initialize pagination and previous section content holder
    continuation = True
    previous_section = None
    revision_count = 0
    
    while continuation:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        
        # Extract the page content and revisions
        pages = data['query']['pages']
        page_id = list(pages.keys())[0]  # Get the first page id
        revisions = pages[page_id].get('revisions', [])

        # Check if any revisions were found
        if not revisions:
            print("No revisions found for this page.")
            return

        # Iterate over the revisions and check for the section
        for rev in revisions:
            slots = rev.get('slots', {})
            main_slot = slots.get('main', {})
            content = main_slot.get('*')  # Ensure we get the content safely

            if content:
                section_text = extract_section(content, section_title)

                if section_text:
                    print(f"Revision ID: {rev['revid']}, Date: {rev['timestamp']}, User: {rev['user']}")
                    print("-" * 80)
                    print(section_text[:500])  # Print first 500 characters of section text
                    print("-" * 80)

                    if previous_section:
                        # Apply and print the diff with the previous version
                        print("Differences with previous version:")
                        print(display_diff(previous_section, section_text))
                        print("-" * 80)

                    # Set the previous section to the current one for comparison in the next iteration
                    previous_section = section_text
                    revision_count += 1

        # Handle continuation
        if 'continue' in data:
            params.update(data['continue'])
        else:
            continuation = False

# Example usage:
page_title = 'Opioid'  # Replace with the page title
section_title = 'Cough'  # Replace with the section you want to track
get_revisions(page_title, section_title)


Revision ID: 704998511, Date: 2016-02-14T22:30:28Z, User: Box73
--------------------------------------------------------------------------------
=
Opioids are generally effective and potent cough suppressants. While many opioids are effective, codeine, hydrocodone and hydromorphone are approved antitussives available in liquid form. Pholcodeine is also used in the UK. Low dose morphine has shown effectiveness relieving chronic cough. Side effects, risk of physical dependence and availability of [[dextromethorphan]] (called a "non-narcotic opioid) limit their common use.
--------------------------------------------------------------------------------
Revision ID: 705003337, Date: 2016-02-14T23:01:17Z, User: Box73
--------------------------------------------------------------------------------
=
Opioids are generally effective and potent [[Cough medicine|cough suppressants]] but are limited by side effects and risk of dependence.<ref>{{Cite journal|title = Therapy for cough: active agent

#### The same with export the dataframe to Excel

In [41]:
# Define the MediaWiki API URL
url = "https://en.wikipedia.org/w/api.php"

# Function to extract a specific section from wikitext using a regex
def extract_section(wikitext, section_title):
    # Use regex to find the section and its content
    section_pattern = re.compile(r"==\s*" + re.escape(section_title) + r"\s*==\s*(.*?)\s*(?=^==|\Z)", re.DOTALL | re.MULTILINE)
    match = section_pattern.search(wikitext)
    if match:
        return match.group(1).strip()
    return None

# Function to display differences between two versions using difflib
def display_diff(text1, text2):
    diff = difflib.unified_diff(
        text1.splitlines(), text2.splitlines(),
        lineterm='', n=2  # Display context lines
    )
    return '\n'.join(diff)

# Function to get all revisions of the page and export to Excel
def get_revisions(page_title, section_title, output_file):
    # Initialize params for the API request
    params = {
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "titles": page_title,
        "rvprop": "ids|timestamp|user|comment|content",
        "rvslots": "main",  # Ensures we get the content from the main slot
        "rvlimit": "max",  # Get all available revisions
        "rvdir": "newer"  # Start from the oldest revisions
    }
    
    # DataFrame to hold revisions data
    revisions_data = []
    
    # Initialize pagination and previous section content holder
    continuation = True
    previous_section = None
    
    while continuation:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        
        # Extract the page content and revisions
        pages = data['query']['pages']
        page_id = list(pages.keys())[0]  # Get the first page id
        revisions = pages[page_id].get('revisions', [])

        # Check if any revisions were found
        if not revisions:
            print("No revisions found for this page.")
            return

        # Iterate over the revisions and check for the section
        for rev in revisions:
            slots = rev.get('slots', {})
            main_slot = slots.get('main', {})
            content = main_slot.get('*')  # Ensure we get the content safely

            if content:
                section_text = extract_section(content, section_title)

                if section_text:
                    diff_text = ""
                    if previous_section:
                        # Apply the diff function
                        diff_text = display_diff(previous_section, section_text)

                    # Only consider revisions where there is a difference
                    if diff_text:
                        print(f"Revision ID: {rev['revid']}, Date: {rev['timestamp']}, User: {rev['user']}")
                        print("-" * 80)
                        print(section_text[:500])  # Print first 500 characters of section text
                        print("-" * 80)
                        print("Differences with previous version:")
                        print(diff_text)
                        print("-" * 80)

                        # Append the data to the list for DataFrame
                        revisions_data.append({
                            "Revision ID": rev['revid'],
                            "Timestamp": rev['timestamp'],
                            "User": rev['user'],
                            "Section Text": section_text[:500],  # Store the first 500 characters of the section text
                            "Differences": diff_text
                        })

                    # Set the previous section to the current one for comparison in the next iteration
                    previous_section = section_text

        # Handle continuation (pagination)
        if 'continue' in data:
            params.update(data['continue'])
        else:
            continuation = False
    
    # Create DataFrame and export to Excel
    if revisions_data:  # Export only if we have relevant data
        df = pd.DataFrame(revisions_data)
        df.to_excel(output_file, index=False)
        print(f"Revisions data exported to {output_file}")
    else:
        print("No relevant changes found for the specified section.")

# Example usage:
page_title = 'Alexander Pushkin'  # Replace with the page title
section_title = 'Social activism'  # Replace with the section you want to track
output_file = "revisions_output.xlsx"  # Define the output file name
get_revisions(page_title, section_title, output_file)

Revision ID: 728041008, Date: 2016-07-02T19:13:54Z, User: 81.159.230.112
--------------------------------------------------------------------------------
=
Pushkin gradually became committed to social reform and emerged as a spokesman for literary radicals. This angered the government, and led to his transfer from the capital in May 1820.<ref>http://russiapedia.rt.com/prominent-russians/literature/aleksandr-pushkin/</ref> He went to the [[Caucasus]] and to the [[Crimean Peninsula|Crimea]], then to [[Kamianka (Cherkasy Oblast)|Kamenka]] and [[Chişinău]], where he became a [[Freemason]].
[[File:Анна Петровна Керн.jpg|left|thumb|180px|Pushkin's marr
--------------------------------------------------------------------------------
Differences with previous version:
--- 
+++ 
@@ -1,4 +1,4 @@
 =
-Pushkin gradually became committed to social reform and emerged as a spokesman for literary radicals. This angered the government, and led to his transfer from the capital (1820). He went to the [[Ca